In [ ]:
import pandas as pd
import numpy as np
import requests
import os
import dotenv
import pvlib

dotenv.load_dotenv()

True

In [ ]:
# Werte zur Eingabe für das ganze Dokument

# latitude = 51.1657
# longitude = 10.4515
# start_date = "2023-01-01"
# end_date = "2023-12-31"


True

In [73]:
from entsoe import EntsoePandasClient

client = EntsoePandasClient(api_key=os.getenv('TOKEN'))

start = pd.Timestamp('20250101', tz='Europe/Brussels')
end = pd.Timestamp('20260101', tz='Europe/Brussels')
country_code = 'DE_LU'  # Belgium
country_code_from = 'DE_LU'  # France
country_code_to = 'DE_LU' # Germany-Luxembourg
type_marketagreement_type = 'A01'
contract_marketagreement_type = "A01"
process_type = 'A44'
resolution = '15min'
sequence = 2

# methods that return Pandas Series
price_series = client.query_day_ahead_prices_local(country_code=country_code,
                                    sequence=sequence,
                                    start=start,
                                    end=end,
                                    resolution=resolution,)

# Umwandeln in einen DF mit entsprechender Spaltenbezeichnung
df_prices = price_series.reset_index()
df_prices = df_prices.resample('h', on='index').mean().reset_index()    # Resampling auf stündlich
df_prices = df_prices.rename(columns={'index':'Datum', 0:'Preis'})
df_prices['Datum'] = df_prices['Datum'].dt.tz_localize(None).dt.tz_localize('UTC')

# Überprüfen der Datentypen und Ausgabe des DataFrames
print(df_prices.dtypes)
print(df_prices)

Datum    datetime64[ns, UTC]
Preis                float64
dtype: object
                         Datum    Preis
0    2025-01-01 00:00:00+00:00  -0.1150
1    2025-01-01 01:00:00+00:00  -0.5150
2    2025-01-01 02:00:00+00:00  -1.4775
3    2025-01-01 03:00:00+00:00  -2.6000
4    2025-01-01 04:00:00+00:00  -6.5000
...                        ...      ...
8756 2025-12-31 20:00:00+00:00  85.0600
8757 2025-12-31 21:00:00+00:00  78.8825
8758 2025-12-31 22:00:00+00:00  76.2825
8759 2025-12-31 23:00:00+00:00  69.2575
8760 2026-01-01 00:00:00+00:00  51.2800

[8761 rows x 2 columns]


In [74]:
# Abruf der relevanten Daten für die Sonneneinstrahlung bei NASA POWER

website_url_meteo = "https://archive-api.open-meteo.com/v1/archive"

# Parameter für den API-Aufruf
params = {
	"latitude": 53.645,
	"longitude": 10.04,
	"start_date": "2025-01-01",
	"end_date": "2025-12-31",
	"hourly": ["shortwave_radiation", "diffuse_radiation", "direct_normal_irradiance"],
}

# API-Anfrage senden
response = requests.get(website_url_meteo, params=params)

# Statuscode der Antwort ausgeben
print("Statuscode:", response.status_code)

# Antwort als JSON in Pandas DF ausgeben (falls erfolgreich)
if response.status_code == 200:
    data = response.json()
    parameter = data['hourly']
    df_meteo = pd.DataFrame({
		'Datum':pd.to_datetime(parameter['time']),
        'Global_Radiation':parameter['shortwave_radiation'],
        'Direct_Radiation':parameter['direct_normal_irradiance'],
        'Diffuse_Radiation':parameter['diffuse_radiation']
	})

    print('Daten erfolgreich im df_meteo gespeichert.')

else:
    print("Fehler beim Abrufen der Daten. Statuscode:", response.status_code)
    print("Fehlertext:", response.text)

# Ausgabe der ersten Zeilen des DataFrames und der Datentypen
print(df_meteo.head())
print(df_meteo.dtypes)

Statuscode: 200
Daten erfolgreich im df_meteo gespeichert.
                Datum  Global_Radiation  Direct_Radiation  Diffuse_Radiation
0 2025-01-01 00:00:00               0.0               0.0                0.0
1 2025-01-01 01:00:00               0.0               0.0                0.0
2 2025-01-01 02:00:00               0.0               0.0                0.0
3 2025-01-01 03:00:00               0.0               0.0                0.0
4 2025-01-01 04:00:00               0.0               0.0                0.0
Datum                datetime64[ns]
Global_Radiation            float64
Direct_Radiation            float64
Diffuse_Radiation           float64
dtype: object


In [52]:
# Abrufen des Sonnenstands mit pvlib
# Standortinformationen
latitude = 53.645
longitude = 10.04

# Zeitstempel für die Berechnung des Sonnenstands
times = pd.date_range(start='2025-01-01', end='2025-12-31', freq='h', tz='UTC')

# Berechnung des Sonnenstands
solpos = pvlib.solarposition.get_solarposition(times, latitude, longitude)

# Umwandeln in DataFrame
df_solar_position = solpos.reset_index()
df_solar_position = df_solar_position.rename(columns={'index':'Datum'})
print(df_solar_position.dtypes)
print(df_solar_position)

Datum                 datetime64[ns, UTC]
apparent_zenith                   float64
zenith                            float64
apparent_elevation                float64
elevation                         float64
azimuth                           float64
equation_of_time                  float64
dtype: object
                         Datum  apparent_zenith      zenith  \
0    2025-01-01 00:00:00+00:00       148.577867  148.577867   
1    2025-01-01 01:00:00+00:00       144.336445  144.336445   
2    2025-01-01 02:00:00+00:00       137.532000  137.532000   
3    2025-01-01 03:00:00+00:00       129.348697  129.348697   
4    2025-01-01 04:00:00+00:00       120.574451  120.574451   
...                        ...              ...         ...   
8732 2025-12-30 20:00:00+00:00       131.433858  131.433858   
8733 2025-12-30 21:00:00+00:00       139.379992  139.379992   
8734 2025-12-30 22:00:00+00:00       145.707023  145.707023   
8735 2025-12-30 23:00:00+00:00       149.154187  149.154187   